<a href="https://colab.research.google.com/github/siddeshnaik/Sentdex_codes/blob/master/Regression_Google_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
!pip install quandl
import quandl

In [ ]:
df=quandl.get("WIKI/GOOGL")

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df=df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df.head()

In [ ]:
df["HL_PCT"]=(df["Adj. High"]-df["Adj. Close"])/df["Adj. Close"]*100
df["PCT_Change"]=(df["Adj. Close"]-df["Adj. Open"])/df["Adj. Open"]*100

In [ ]:
df.head()

In [ ]:
df =df[["Adj. Close","HL_PCT","PCT_Change","Adj. Volume"]]

In [ ]:
df.head()

In [ ]:
forcaste_col="Adj. Close"

In [ ]:
df.fillna(-99999, inplace = True)

In [ ]:
import math 
forcast_out= int(math.ceil(0.01*len(df)))
forcast_out

In [ ]:
df["labels"]=df[forcaste_col]

In [ ]:
df

In [ ]:
df["labels"]=df[forcaste_col].shift(-forcast_out)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

In [ ]:
from sklearn import preprocessing, svm
# from sklearn.ensemble import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
X=np.array(df.drop(["labels"], axis =1))
X = X[:-forcast_out]
X_lately = X[-forcast_out:]
X = preprocessing.scale(X)
df.dropna(inplace=True)
y= np.array(df["labels"])
y = y[:-forcast_out]


In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [ ]:
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train,y_train)
with open('linearregression.pickle','wb') as f:
    pickle.dump(clf, f)

In [ ]:
accuracy = clf.score(X_test,y_test)
accuracy

In [ ]:
clf_1 = svm.SVR()
clf_1.fit(X_train,y_train)
with open('SVR.pickle','wb') as f:
    pickle.dump(clf_1, f)
accuracy_1 = clf.score(X_test,y_test)
accuracy_1

In [ ]:
prediction_val=clf.predict(X_lately)

In [ ]:
print("Predicion:",prediction_val,"\n\n","Accuracy_LinearRegression :",acccuracy,"\n\n","Forcasting period:",forcast_out)

In [ ]:
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

In [ ]:
df['Forecast'] = np.nan
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day

In [ ]:
for i in prediction_val:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += 86400
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)]+[i]

In [ ]:
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
pickle_in = open('linearregression.pickle','rb')
clf = pickle.load(pickle_in)
print(clf.predict(X_lately))
print(clf.score(X_test,y_test))

In [ ]:
pickle_in = open('SVR.pickle','rb')
clf_1 = pickle.load(pickle_in)
print(clf_1.predict(X_lately))
print(clf_1.score(X_test,y_test))